In [3]:
import pandas as pd 
import sys
import glob 
import os
import tqdm
## ---------- Define Root Path ---------- ## 
ROOTPATH = '/Users/zhenggong/Documents/Github/IBES_selected/' # for importing and reference management 
sys.path.append(ROOTPATH)

# Logic of the broker matching between IBES broker abbr. (ESTIMID) and CapitalIQ estimatebroker

remark: use ESTIMID to represent brokers, instead of EMASKCD 

1. start with IBES load in the data of recommendation (upgrade, downgrade, maintain ...)
2. limit the scope of IBES data to recommendations on US-firm by USFIRM=1.0
3.  

In [2]:
# recommendataions (buy or sell or HOLD)

# Focus on IRECCD (which is reconciled at IBES level) 
# etext and ereccd are explicit recommendation (can be less comparable)
recommend = pd.read_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/IBES_csv/nahghdromygr63ao.csv', index_col=[0]).reset_index(drop=True)
# length of the data
print('The length of the data is: ' + str(len(recommend)))

recommend.head(5)


/var/folders/0j/2k9yrkz575s8zxhdwtq0yrvm0000gn/T/ipykernel_65564/77959839.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  recommend = pd.read_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/IBES_csv/nahghdromygr63ao.csv', index_col=[0]).reset_index(drop=True)


The length of the data is: 3142392


,TICKER,CUSIP,CNAME,OFTIC,ACTDATS,ESTIMID,ANALYST,ERECCD,ETEXT,IRECCD,ITEXT,EMASKCD,AMASKCD,USFIRM,ACTTIMS,REVDATS,REVTIMS,ANNDATS,ANNTIMS
0,0000,87482X10,TALMER BANCORP,TLMR,2014-03-10,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2.0,BUY,659.0,71182.0,1.0,1960-01-01 08:54:02.999,2016-01-26,1960-01-01 09:35:52.000,2014-03-10,1960-01-01 00:20:00
1,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,JPMORGAN,ALEXOPOULOS S,NaN,OVERWEIGHT,2.0,BUY,1243.0,79092.0,1.0,1960-01-01 17:10:47.000,2016-01-26,1960-01-01 10:09:34.000,2014-03-10,1960-01-01 00:25:00
2,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,KEEFE,MCGRATTY C,2,OUTPERFORM,2.0,BUY,1308.0,119962.0,1.0,1960-01-01 15:17:15.000,2015-07-30,1960-01-01 07:25:58.000,2014-03-09,1960-01-01 17:05:00
3,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,RAYMOND,LONG D,1,STRONG BUY,1.0,STRONG BUY,1929.0,80474.0,1.0,1960-01-01 15:49:22.000,2016-01-19,1960-01-01 08:06:44.000,2014-03-10,1960-01-01 06:48:00
4,0000,87482X10,TALMER BANCORP,TLMR,2014-03-11,STERNE,GREEN P,2,NEUTRAL,3.0,HOLD,2209.0,50789.0,1.0,1960-01-01 15:22:08.000,2014-11-07,1960-01-01 12:20:17.000,2014-03-10,1960-01-01 07:45:00


In [3]:
# restrict to only brokerage firms that comments are USFIRM
recommend = recommend.query('USFIRM == 1.0').sort_values(by=['ACTDATS'])
# length of the data
print('The length of the data is: ' + str(len(recommend)))


The length of the data is: 842183


In [4]:
# clean out the weird ANALYST NAME
recommend = recommend.dropna(subset='ANALYST')
recommend = recommend[~recommend["ANALYST"].str.contains("RESEARCH")] # filter out RESEARCH, RESEARCH DEPT
# filter out the recommendation with ANALYST no space in between
recommend = recommend[recommend["ANALYST"].str.contains(" ")]

# filter out old data
recommend = recommend.query("ANNDATS > '2000-01-01'")
print('The length of the data is: ' + str(len(recommend)))

The length of the data is: 629940


In [5]:
unique_brokers = recommend.drop_duplicates(subset='AMASKCD')['ESTIMID'].groupby(recommend['ESTIMID']).count().sort_values(ascending=False)
unique_brokers = pd.DataFrame(unique_brokers).query('ESTIMID >= 1') # we do not filter out any broker no mateer how many analysts they have

# all the brokers that have more than 10 analysts
unique_brokers

,ESTIMID
ESTIMID,
MERRILL,1206
JPMORGAN,628
GOLDMAN,559
MORGAN,525
FBOSTON,506
...,...
ENERGO,1
SANTANDP,1
ENAM,1


# look at this this is a problem
   what represents EMASKCD, what represents ESTIMID
   They are not ONE TO ONE!

In [43]:
recommend.query('EMASKCD == 1534.0').drop_duplicates(subset='ESTIMID')
# recommend.query('ESTIMID == "JPMORGAN"').drop_duplicates(subset='EMASKCD')

,TICKER,CUSIP,CNAME,OFTIC,ACTDATS,ESTIMID,ANALYST,ERECCD,ETEXT,IRECCD,ITEXT,EMASKCD,AMASKCD,USFIRM,ACTTIMS,REVDATS,REVTIMS,ANNDATS,ANNTIMS
3036433,TOM,G8915Z10,HILFIGER TOMMY C,TOM,2000-01-03,FBOSTON,ROSENBERG D,B,BUY,2.0,BUY,846.0,1342.0,1.0,1960-01-01 07:59:40.000,2000-04-06,1960-01-01 15:58:37.000,2000-01-03,1960-01-01 07:59:40.000
2862256,PPP,73044810,POGO PRODUCG,PPP,2000-01-03,FBOSTON,PACE P,B,BUY,2.0,BUY,846.0,4221.0,1.0,1960-01-01 16:27:42.000,2000-05-18,1960-01-01 14:47:03.000,2000-01-03,1960-01-01 16:27:42.000
2687849,LSTR,51509810,LANDSTAR SYSTEM,LSTR,2000-01-03,FBOSTON,YABLON G,SB,STRONG BUY,1.0,STRONG BUY,846.0,1549.0,1.0,1960-01-01 07:59:28.000,2002-09-06,1960-01-01 18:19:04.000,2000-01-03,1960-01-01 07:59:28.000
2667108,LBOR,50540120,LABOR READY,LRW,2000-01-04,FBOSTON,WALDO A,H,HOLD,3.0,HOLD,846.0,43879.0,1.0,1960-01-01 09:11:46.000,2002-02-22,1960-01-01 16:06:51.000,2000-01-04,1960-01-01 09:11:46.000
2202090,ALI,02146C10,ALTERRA HEALTH,ALI,2000-01-04,FBOSTON,SWENSON J,H,HOLD,3.0,HOLD,846.0,31952.0,1.0,1960-01-01 15:14:12.000,2002-03-13,1960-01-01 17:49:24.000,2000-01-04,1960-01-01 15:14:12.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876760,PTVE,69526K10,PACTIV EVERGREEN,PTVE,2023-09-13,FBOSTON,WOODWORTH C,10,OUTPERFORM,2.0,BUY,846.0,84007.0,1.0,1960-01-01 07:51:29.000,2023-09-13,1960-01-01 07:51:29.000,2023-09-13,1960-01-01 06:02:00.000
2327695,CCK,22836810,CROWN HOLDINGS,CCK,2023-09-13,FBOSTON,WOODWORTH C,10,OUTPERFORM,2.0,BUY,846.0,84007.0,1.0,1960-01-01 07:37:52.000,2023-09-13,1960-01-01 07:37:52.000,2023-09-13,1960-01-01 06:02:00.000
2804331,OI1,67098H10,O-I GLASS,OI,2023-09-13,FBOSTON,WOODWORTH C,11,NEUTRAL,3.0,HOLD,846.0,84007.0,1.0,1960-01-01 07:37:56.000,2023-09-13,1960-01-01 07:37:56.000,2023-09-13,1960-01-01 06:02:00.000
10209,01IF,93627C10,WARRIOR MET COAL,HCC,2023-09-15,FBOSTON,WOODWORTH C,10,OUTPERFORM,2.0,BUY,846.0,84007.0,1.0,1960-01-01 06:55:21.000,2023-09-15,1960-01-01 06:55:21.000,2023-09-15,1960-01-01 00:00:00


We need to get rid of Asian Last names as they are not well dealt with (last name, first name confusion)

In [30]:
# thoese are by gpt4o
# prompt:
# give me a list of 100 common Taiwanese or Malaysian last names, sorted alphabetically, stored in the variable common_taiwan_malaysian_last_names:

common_chinese_names = [
    "Ai", "An", "Bai", "Bao", "Bi", "Bo", "Bu", "Cao", "Cai", "Cen",
    "Ceng", "Chen", "Cheng", "Chai", "Chang", "Chu", "Cui", "Deng", "Ding", "Dong",
    "Du", "Fang", "Feng", "Fu", "Gao", "Gu", "Guan", "Guo", "Han", "Hao",
    "He", "Hong", "Hou", "Hu", "Huang", "Huo", "Ji", "Jiang", "Jin", "Kong",
    "Kou", "Kuang", "Lei", "Liang", "Lian", "Liao", "Lin", "Liu", "Lou", "Long",
    "Luo", "Lu", "Ma", "Mao", "Meng", "Mo", "Mu", "Nie", "Ni", "Ouyang",
    "Pan", "Peng", "Qian", "Qiao", "Qin", "Qiu", "Qu", "Ran", "Rao", "Ren",
    "Shao", "Shen", "Shi", "Shu", "Song", "Su", "Sun", "Tan", "Tao", "Tian",
    "Tong", "Wang", "Wei", "Wu", "Xi", "Xiao", "Xie", "Xing", "Xiong", "Xu",
    "Xue", "Yan", "Yang", "Yao", "Ye", "Yin", "Ying", "Yu", "Yuan", "Zeng",
    "Zhang", "Zhao", "Zhan", "Zhan", "Zheng", "Zhou", "Zhu", "Zou"
]
common_korean_last_names = [
    "Ahn", "Baek", "Bae", "Byun", "Cha", "Chae", "Chang", "Cheon", "Choi", "Chun", "Chung", "Do", "Eo", "Go", "Gong", "Goo", "Gwak", "Ha", "Han", "Heo", "Hong", "Hwang", "Im", "Jang", "Je", 
    "Jeon", "Jo", "Joo", "Ju", "Jung", "Kang", "Kang", "Kim", "Ko", "Kwon", "Lee", "Lim", "Ma", "Moon", "Na", "Nam", "Noh", "Oh", "Park", "Pyeon", "Rhee", "Rim", "Seo", "Seok", "Seol", "Shin", 
    "Shon", "Sim", "Son", "Song", "Suh", "Suk", "Sung", "Yoo", "Yoon", "You", "Woo", "Yang", "Yeo", "Yeom", "Yeon", "Yoon", "Yu", "Yun", "Ahn", "Baek", "Bae", "Byun", "Cha", "Chae", "Chang", 
    "Cheon", "Choi", "Chun", "Chung", "Do", "Eo", "Go", "Gong", "Goo", "Gwak", "Ha", "Han", "Heo", "Hong", "Hwang", "Im", "Jang", "Je", "Jeon", "Jo", "Joo", "Ju", "Jung"
]
common_japanese_last_names = [
    "Abe", "Aoki", "Chiba", "Eguchi", "Endo", "Fujii", "Fujimoto", "Fujita", "Fukuda", "Goto", "Hara", "Hasegawa", "Hashimoto", "Hayashi", "Hirano", "Igarashi", "Ikeda", "Imai", "Inoue", 
    "Ishida", "Ishii", "Ishikawa", "Ito", "Kawaguchi", "Kawamoto", "Kikuchi", "Kimura", "Kinoshita", "Kishimoto", "Kita", "Kitagawa", "Kobayashi", "Kondo", "Koyama", "Kubo", "Maeda", 
    "Maruyama", "Masuda", "Matsuda", "Matsui", "Matsumoto", "Miyamoto", "Miyazaki", "Mori", "Murakami", "Nakagawa", "Nakamura", "Nakano", "Nakashima", "Nakayama", "Nishida", "Nishimura", 
    "Noguchi", "Nomura", "Oba", "Ohashi", "Okada", "Okamoto", "Ono", "Ota", "Ozaki", "Saito", "Sakai", "Sakamoto", "Sato", "Shibata", "Shimada", "Sugawara", "Sugimoto", "Sugiyama", 
    "Suzuki", "Takahashi", "Takaki", "Takano", "Takeuchi", "Tanaka", "Taniguchi", "Tani", "Tateishi", "Tokunaga", "Tomita", "Tsuda", "Uchida", "Ueda", "Ueno", "Umeda", "Wada", "Watanabe", 
    "Yamamoto", "Yamaguchi", "Yamashita", "Yamazaki", "Yasuda", "Yokoyama", "Yoshida", "Yoshikawa", "Yoshimura", "Yoshino"
]
common_taiwan_malaysian_last_names = [
    "Ang", "Au", "Chan", "Chang", "Cheah", "Chee", "Chen", "Cheng", "Cheong", "Chew", "Chia", "Chin", "Chong", "Choo", "Chua", "Chung", 
    "Fong", "Foo", "Gan", "Goh", "Ho", "Hoh", "Hong", "Hoo", "Huang", "Kang", "Khaw", "Khoo", "Kok", "Koo", "Kua", "Kwan", "Kwok", 
    "Lai", "Lam", "Lau", "Law", "Lee", "Leong", "Lim", "Ling", "Liu", "Loh", "Low", "Ma", "Mah", "Mok", "Ng", "Ngai", "Ngoo", "Nio", 
    "Ong", "Ow", "Pang", "Poon", "Quek", "Sim", "Soo", "Su", "Tan", "Tang", "Teh", "Teng", "Teo", "Tey", "Tham", "Toh", "Tsai", "Tse", 
    "Wai", "Wang", "Wee", "Weng", "Wong", "Woo", "Yap", "Yau", "Yeo", "Yeoh", "Yew", "Yoong", "Yong", "Yuen", "Yung", "Zhang", "Zhao", 
    "Zheng", "Zhou", "Zhu"
]

manual_add = ['Li', 'Ge', 'Chuen', 'Lv', 'Yip', 'Qiang', 'Kwai', 'Miao', 'Quan', 'ZUO', 'LUI', 'Fan', 'Duan', 'Wan', 'Huangfu', 'Zhang', 'Teng', 'Po', 'Fei', 'Chian', 'Chiang', 'Yen', 'Lieu', 'Fung', 'Zhong', 'Wen']

# get the set of the lists
joint_lastname_to_del = common_chinese_names + common_korean_last_names + common_japanese_last_names + common_taiwan_malaysian_last_names + manual_add

# get upper case of list of string
joint_lastname_to_del = [x.upper() for x in joint_lastname_to_del]
# joint_lastname_to_del

recommend['asian_flag'] = False
for index, row in recommend.iterrows():
    _last_name_temp = row['ANALYST'].split(' ')[0]
    if _last_name_temp in joint_lastname_to_del:
        # print(_last_name_temp)
        recommend.loc[index, 'asian_flag'] = True



recommend = recommend.query('asian_flag == False')

/var/folders/0j/2k9yrkz575s8zxhdwtq0yrvm0000gn/T/ipykernel_55342/1554474301.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend['asian_flag'] = False


# Now we try to look at the broker one by one and try to match them by some ways

In [10]:
for broker in unique_brokers.index:
    broker_record = recommend.query('ESTIMID == @broker')
    broker_record = broker_record['ANALYST'].groupby(broker_record['ANALYST']).count().sort_values(ascending=False)
    broker_record = broker_record.head(100)
    
    name = []
    initial = []
    lastname = []
    for analyst in broker_record.index:
        name.append(analyst)
        initial.append(analyst[-1])
        lastname.append(analyst.split(' ')[0])
    
    name_df = pd.DataFrame({'Name': name, 'Initial': initial, 'Lastname': lastname})
    # lastname drop " " and ,
    name_df['Lastname_SQL_fuzzy'] = name_df['Lastname'].apply(lambda x: x + '%' if len(x) >= 8 else x).apply(lambda x: x.replace('"', '')).apply(lambda x: x.replace(',', ''))
    name_df['Firstname_SQL_fuzzy'] = name_df['Initial'].apply(lambda x: x + '%').apply(lambda x: x.replace('"', '')).apply(lambda x: x.replace(',', ''))

    print(name_df)
    name_df.to_csv('/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_with_analyst_employed_us/' + broker + '.csv', index=False)


               Name Initial       Lastname Lastname_SQL_fuzzy  \
0   HOEXTER       K       K        HOEXTER            HOEXTER   
1   MOREEN        G       G         MOREEN             MOREEN   
2   DUMOULIN-SMIT J       J  DUMOULIN-SMIT     DUMOULIN-SMIT%   
3   SAKWA         S       S          SAKWA              SAKWA   
4   BRUCE         K       K          BRUCE              BRUCE   
..              ...     ...            ...                ...   
95  BUCKLEY       J       J        BUCKLEY            BUCKLEY   
96  BALSKY        H       H         BALSKY             BALSKY   
97  CHOI          K       K           CHOI               CHOI   
98  NAGLE         C       C          NAGLE              NAGLE   
99  KRAUSHAAR     J       J      KRAUSHAAR         KRAUSHAAR%   

   Firstname_SQL_fuzzy  
0                   K%  
1                   G%  
2                   J%  
3                   S%  
4                   K%  
..                 ...  
95                  J%  
96                 

# Now let's try to call in CapitalIQ database

In [5]:
ROOTPATH = '/Users/zhenggong/Documents/Github/IBES_selected/capitaliq' # for importing and reference management 
import sys
sys.path.append(ROOTPATH)
import pandas as pd 
from cfg import SERVER_TIMEZONE,DBINFO
import json
import psycopg2
from datetime import datetime, timedelta


def read_sql_to_df(sql, db, cursor):
    """Internal: execute sql and get dataframe as a return 
    
    Args:
        sql (str): sql to be executed 
        db (psycopg2.connect): connect to ciq target database 
        cursor (database connection.cursor): 
    
    Returns:
        pd.DataFrame: contains result for executed sql 
    """
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(data,columns = columns)
    except (Exception, psycopg2.DatabaseError) as error:
        db.rollback()
        print("Error: %s" % error)


def get_connection(dbInfo):
    """get the db connection using info in dbInfo json file
    
    Args:
        dbInfo (.json): user-pw for ciq target database 
    
    Returns:
        psycopg2.connect
    """
    with open(dbInfo,'r') as f:
        u = json.load(f)

    db = psycopg2.connect(
        host=u['host'],
        database=u["database"],
        user=u["user"],
        password=u["pwd"])
    return db

###################################################################################
def get_estimateanalyst(names, connection = None):

    ciq_broker_lists = []
    for _, name in names.iterrows():
        lastname = name['Lastname_SQL_fuzzy']
        firstname = name['Firstname_SQL_fuzzy']
        # print(lastname, firstname)

        sql = f"""
        SELECT * from ciqestimateanalyst where upper(firstname) like '{firstname}' and upper(lastname) like $${lastname}$$;
        """
        if connection is None:
            connection = get_connection(DBINFO)
        cursor = connection.cursor()

        exec_res = list(read_sql_to_df(sql, connection, cursor)['estimatebrokerid'].unique())
        # if there is no such analyst, no need to append
        if len(exec_res) == 0: 
            continue
        ciq_broker_lists.append(exec_res)
    
    return ciq_broker_lists

def get_estimatebrokername(ids, connection = None):

    sql = f"""
    SELECT * from ciqestimatebroker where estimatebrokerid in ({', '.join([str(id) for id in ids])}) ;
    """
    if connection is None:
        connection = get_connection(DBINFO)
    cursor = connection.cursor()

    return read_sql_to_df(sql, connection, cursor)

###################################################################################

def flatten(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten(item))
        else:
            flattened_list.append(item)
    return flattened_list


In [12]:
IBES_NAME_FILES = glob.glob(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_with_analyst_employed_us/*.csv')

for IBES_NAME_FILE in tqdm.tqdm(IBES_NAME_FILES):

    res = get_estimateanalyst(pd.read_csv(IBES_NAME_FILE, index_col=[0]))
    # keep right before .csv
    IBES_NAME = IBES_NAME_FILE.split('/')[-1][:-4]

    flattened = flatten(res)
    flattened = pd.DataFrame(flattened, columns=['estimatebrokerid'])

    df = pd.DataFrame(flattened['estimatebrokerid'].value_counts())
    df.to_csv(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/BROKER_MATCHING_PROCESSED_US/{IBES_NAME}.csv')

100%|██████████| 940/940 [13:54<00:00,  1.13it/s]


In [ ]:
# legacy code
overall_res = []
 
if len(df) == 0:
    continue
total_valid_record = len(res)
confidence_on_density = df['count'].values[0] /  total_valid_record# the density 
try:
    confidence_on_distance = (df['count'].values[0] - df['count'].values[1]) / df['count'].values[0] # the difference between the first and second
except:
    confidence_on_distance = -1

choice = df.index[0]

# append as dictionary to final result
content = {'IBES_NAME': IBES_NAME,
        'total_valid_record': total_valid_record, 
        'confidence_on_density': confidence_on_density,
    'confidence_on_distance': confidence_on_distance,
    'choice': choice}
overall_res.append(content)
print(content)


In [25]:
import openai
import time

# Function to get an integer response from OpenAI API
def get_integer_from_openai(prompt):
    openai.api_key = os.getenv("OPENAI_KEY")
    
    max_retries = 10
    retries = 0
    while retries < max_retries:
        try:
            # Call the OpenAI API
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )
            # Extract the text response from the API
            response_text = int(response.choices[0].message.content)
            break
        except Exception as e:
            retries += 1
            print('maxing out retries for the following error: ' + str(e))
            time.sleep(1)

    return response_text



In [65]:
broker_matching_name = 'ACIRESCA'

broer_matching_names = glob.glob('/Users/zhenggong/Documents/GitHub/IBES_selected/data/BROKER_MATCHING_PROCESSED_US/*.csv')
broer_matching_names = [x.split('/')[-1][:-4] for x in broer_matching_names]

for broker_matching_name in tqdm.tqdm(broer_matching_names):
    # print(broker_matching_name)
    if os.path.exists(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_matching_result_v2/{broker_matching_name}.csv'):
        continue

    broker_matching_df = pd.read_csv(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/BROKER_MATCHING_PROCESSED_US/{broker_matching_name}.csv', index_col=[0])
    if len(broker_matching_df) == 0:
        continue
    else:
        broker_name = get_estimatebrokername(broker_matching_df.index)

        broker_matching_df = pd.merge(broker_matching_df, broker_name, left_index=True, right_on='estimatebrokerid', how='inner')
        # print(broker_matching_df.head(5))

        if len(broker_matching_df) == 1:
            broker_matching_df['choice_id'] = broker_matching_df['estimatebrokerid'].values
            broker_matching_df['gpt'] = False

        else:
            # use relative strength to decide 
            if broker_matching_df['count'].values[0] / broker_matching_df['count'].values[1] >= 4: 
                broker_matching_df['choice_id'] = broker_matching_df['estimatebrokerid'].values[0]
                broker_matching_df['gpt'] = False
            else:
                old_match_table = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
                            sheet_name='Old match table').dropna(subset=['Baname'])
                broker_oldmatch1 = old_match_table.query('Baid == @broker_matching_name')['Baname'].values

                europe_table = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
                            sheet_name='Europe').dropna(subset=['Group name'])
                broker_oldmatch2 = europe_table.query('ESTIMID == @broker_matching_name')['Group name'].values

                # merge two lists 
                if len(broker_oldmatch2) == 0:
                    broker_oldmatch = broker_oldmatch1
                else:
                    broker_oldmatch = broker_oldmatch2

                broker_transition_table = pd.read_excel(open('/Users/zhenggong/Documents/GitHub/IBES_selected/data/Broker codes collection v2.xlsx', 'rb'),
                            sheet_name='Top group table')
                broker_transition_table.columns = ['sub', 'top', 'date1', 'date2']

                if len(broker_oldmatch) == 0:
                    broker_transition_info = []
                else:
                    broker_transition_info = broker_transition_table[broker_transition_table["sub"].astype(str).str.contains('|'.join(broker_oldmatch))]['top'].values


                _dict = broker_matching_df[['brokername', 'estimatebrokerid', 'count']].head(10).to_dict('records')
                _extra_name_info = list(set(list(broker_transition_info) + list(broker_oldmatch)))
                if len(_extra_name_info) == 0:
                    _extra_name_info = ''
                else:
                    _extra_name_info = 'The IBES Broker name is: ' + str(_extra_name_info) + ' .'

                prompt = f"""
                You are a stock analyst, the task is to match the IBES Broker with the Capital IQ Broker.
                The IBES Broker code is '{broker_matching_name}'.  {_extra_name_info}
                The potential Capital IQ Broker can be one of the brokername: {_dict}.

                In the dictionary given, the count is a predefined measure of the confident of the match of the brokername.

                Tell me which one of the brokerid is the most likely match for the IBES Broker code '{broker_matching_name}'.
                
                return an integer indicating the brokerid and nothing else! 
                if not sure return 0 and nothing else"""
                # print(prompt)

                result = get_integer_from_openai(prompt)

                broker_matching_df['choice_id'] = result
                broker_matching_df['gpt'] = True

    broker_matching_df.to_csv(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_matching_result_v2/{broker_matching_name}.csv', index=False) 
            

    

 75%|███████▍  | 701/940 [00:01<00:00, 520.25it/s]/var/folders/0j/2k9yrkz575s8zxhdwtq0yrvm0000gn/T/ipykernel_65679/4043888826.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  broker_transition_info = broker_transition_table[broker_transition_table["sub"].astype(str).str.contains('|'.join(broker_oldmatch))]['top'].values
 77%|███████▋  | 725/940 [00:18<00:07, 28.35it/s] 

maxing out retries for the following error: invalid literal for int() with base 10: "The broker most likely to match the IBES Broker code 'KGIFT' is 796."


 87%|████████▋ | 817/940 [01:26<01:17,  1.58it/s]

maxing out retries for the following error: invalid literal for int() with base 10: "The most likely match for the IBES Broker code 'LAIDCOM' is 3027."


 90%|████████▉ | 844/940 [01:49<01:14,  1.29it/s]/var/folders/0j/2k9yrkz575s8zxhdwtq0yrvm0000gn/T/ipykernel_65679/4043888826.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  broker_transition_info = broker_transition_table[broker_transition_table["sub"].astype(str).str.contains('|'.join(broker_oldmatch))]['top'].values
100%|██████████| 940/940 [03:10<00:00,  4.94it/s]


In [67]:
# aggregate the results 
broker_matching_names = glob.glob('/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_matching_result_v2/*.csv')
broker_matching_names = [x.split('/')[-1][:-4] for x in broker_matching_names]

res = []
for broker_matching_name in broker_matching_names:
    # print(broker_matching_name)
    df = pd.read_csv(f'/Users/zhenggong/Documents/GitHub/IBES_selected/data/broker_matching_result_v2/{broker_matching_name}.csv').iloc[0]
    choice = df['choice_id']
    gpt = df['gpt']
    # print(df)
    res.append({'broker_matching_name': broker_matching_name, 'choice': choice, 'gpt': gpt})

res = pd.DataFrame(res).sort_values(by=['broker_matching_name'])

broker_name = get_estimatebrokername(res['choice'].values)

res = pd.merge(res, broker_name, left_on='choice', right_on='estimatebrokerid', how='left').drop(columns=['estimatebrokerid', 'companyid'])
print(res)

res.to_csv('broker_match_v2.csv')


    broker_matching_name  choice    gpt  \
0                    ABG     229  False   
1                ABLEICH      34  False   
2               ABNLATAM       0   True   
3                   ABSA    1859   True   
4                ACCIVAL       3  False   
..                   ...     ...    ...   
855             ZACKSINV    1356   True   
856               ZANNEX     206   True   
857               ZELMAN       0   True   
858                ZOETE       4   True   
859              ZURCHER    1100  False   

                                     brokername  
0                            ABG Sundal Collier  
1                     Natixis Bleichroeder Inc.  
2                                           NaN  
3                             Absa Bank Limited  
4                                 Citigroup Inc  
..                                          ...  
855              Zacks Investment Research Inc.  
856  Excellence Nessuah Brokerage Services Ltd.  
857                              